In [2]:
import pandas as pd

In [3]:
!pip install razdel
!pip install pymorphy2

     |████████████████████████████████| 61kB 3.7MB/s 
     |████████████████████████████████| 8.2MB 6.6MB/s 


In [4]:
import re
import string

import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords
from razdel import tokenize
import pymorphy2
import pandas as pd

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [5]:
nltk.download('averaged_perceptron_tagger_ru')

[nltk_data] Downloading package averaged_perceptron_tagger_ru to
[nltk_data]     /root/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger_ru.zip.


True

In [8]:
df = pd.read_csv('/content/female_short.csv', sep=';', error_bad_lines=False)

In [9]:
df1 = pd.read_csv('/content/db1.csv', sep=';', error_bad_lines=False)

In [10]:
df_f = pd.read_csv('/content/fe_short.csv', sep=';', error_bad_lines=False)

In [13]:
df = pd.concat([df, df_f, df1])

In [ ]:
df.to_csv('database.csv')

In [11]:
stopwords_ru = stopwords.words('russian')

def make_tokens(text):
    tokens = []
    text = text.lower()
    text = re.sub(r"[^а-яё]", ' ', text) 
    for token in tokenize(text):
        word = list(token)[-1]
        if word not in stopwords_ru:
            tokens.append(word)
    return tokens

def tagger(text):
    tokens = make_tokens(text)
    tokens = nltk.pos_tag(tokens, lang='rus')
    return tokens

In [ ]:
def preprocess():  
  base = df['Lyrics'].values.tolist()
  new_lems = []
  new_tok = []
  for i in base:
    lemmas = tagger(str(i))
    tokens = make_tokens(str(i))
    new_lems.append(lemmas)
    new_tok.append(tokens)
  df['tokens'] = new_tok
  df['tagged'] = new_lems
  return df
preprocess()

In [15]:
df.to_csv('full_data.csv')

In [16]:
data = df.copy()
data = data.drop(columns=['Title', 'Artist', 'Column1', 'processed', 'tokens'])

In [17]:
res = []
for i in data.tagged.values:
  a = [list(m) for m in i]
  res.append(a)

In [ ]:
res

In [19]:
songs = []
for i in res:
  l = []
  for m in i:
    if m[1] == 'V':
      m[0] = 'MASK'
    l.append(m[0])
  songs.append(l)

In [77]:
more = pd.read_csv('/content/result.csv', sep=',', error_bad_lines=False)

In [ ]:
more.drop(columns='Unnamed: 0')

In [20]:
data['gender'] = data['gender'].astype('int32')

In [21]:
data['gender'].dtype

dtype('int32')

In [22]:
data['tokens'] = songs
data['tokens'] = data['tokens'].apply(lambda x: " ".join(x))

In [ ]:
data.drop(columns=['Lyrics', 'tagged'])

In [24]:
columns_titles = ['tokens', 'gender']
data=data.reindex(columns=columns_titles)

In [ ]:
data = pd.read_csv('/content/clean_data (1).csv', sep=',', error_bad_lines=False)

In [25]:
text = data.tokens.values
gender = data.gender.values

In [ ]:
data.drop(columns='Unnamed: 0')

In [ ]:
data.to_csv('result.csv')

In [ ]:
!pip install numpy git+https://github.com/makcedward/nlpaug.git

In [ ]:
!pip install transformers

In [29]:
import torch

In [ ]:
!pip install fairseq

In [31]:
import nlpaug.augmenter.char as nac
import nlpaug.augmenter.word as naw
import nlpaug.augmenter.sentence as nas
import nlpaug.flow as nafc

from nlpaug.util import Action

In [41]:
def augment(text):
  augmented = []
  aug = naw.RandomWordAug(action="swap")
  for i in text:
    augmented_text = aug.augment(i)
    print("Original:")
    print(i)
    print("Augmented Text:")
    print(augmented_text)
    augmented.append(augmented_text)
  return augmented

In [ ]:
first = augment(text)

In [ ]:
second = augment(first)

In [45]:
aug_df = pd.DataFrame(columns=['tokens', 'gender'])

In [46]:
aug_df1 = pd.DataFrame(columns=['tokens', 'gender'])

In [47]:
aug_df['tokens'] = first
aug_df['gender'] = gender

In [48]:
aug_df1['tokens'] = second
aug_df1['gender'] = gender

In [ ]:
aug_df

In [51]:
more_data = pd.concat([aug_df, data, aug_df1])

In [ ]:
more_data

In [53]:
more_data.to_csv('more_data1.csv')